In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install bertopic

env: TOKENIZERS_PARALLELISM=false
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 69.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819467 sha256=72d345bb25260048517f96ed879d7e1c2b4015d9635cc5b77593ab05ca9fc533
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=ad7725dd6dca5787d33bad4f288ff988b17bb3d07356cdf24f6a8e88a6575138
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/

In [2]:
import pandas as pd
# we only use the text fields so let's not load anything else
df = pd.read_csv(filepath_or_buffer='/kaggle/input/stranger-things-dialogue-dataset/stranger_things_all_dialogue.csv',
                usecols=['raw_text', 'stage_direction', 'dialogue'])
# make a big text field that includes everything
df['text'] = df['raw_text'] + ' ' + df['stage_direction'] + ' ' + df['dialogue'].fillna(value='')
df = df.dropna(subset=['text'])
# try to remove blank tokens
df['text'] = df['text'].apply(func=lambda x: ' '.join(x.split()))
df.head()

,raw_text,stage_direction,dialogue,text
0,[crickets chirping],[crickets chirping],NaN,[crickets chirping] [crickets chirping]
1,[alarm blaring],[alarm blaring],NaN,[alarm blaring] [alarm blaring]
2,[panting],[panting],NaN,[panting] [panting]
3,[elevator descending],[elevator descending],NaN,[elevator descending] [elevator descending]
4,[elevator dings],[elevator dings],NaN,[elevator dings] [elevator dings]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10678 entries, 0 to 32518
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   raw_text         10678 non-null  object
 1   stage_direction  10678 non-null  object
 2   dialogue         4594 non-null   object
 3   text             10678 non-null  object
dtypes: object(4)
memory usage: 417.1+ KB


In [4]:
from collections import Counter
text = ' '.join(df['text'].dropna().values).replace('[', '').replace(']', '')
counts = Counter(text.split())
print(counts.most_common(n=20))
unique = set(text.split())
print(len(unique))

[('music', 1801), ('playing', 1392), ('I', 960), ('you', 782), ('in', 769), ('sighs', 764), ('grunts', 637), ('the', 594), ('chuckles', 548), ('a', 522), ('to', 510), ('continues', 478), ('panting', 477), ('Russian', 476), ('Dustin', 458), ('Hopper', 418), ('Mike', 408), ('on', 377), ('Joyce', 368), ('is', 345)]
7127


Some of our most common/prevalent tokens are coming from the stage directions, which is weird.

In [5]:
from collections import defaultdict
document_counts = defaultdict(int)
for index, row in df.dropna(subset=['text']).iterrows():
    current = set(row['text'].replace('[', '').replace(']', '').split())
    for token in current:
        document_counts[token] += 1
percentages = {key: value/len(df['text'].dropna()) for key, value in document_counts.items()}
percentages = sorted(list(percentages.items()), reverse=True, key=lambda x: x[1])
print(percentages[:10])

[('music', 0.08540925266903915), ('playing', 0.06518074545795093), ('I', 0.043079228319910096), ('sighs', 0.038209402509833304), ('in', 0.03568083910844728), ('you', 0.03511893613036149), ('grunts', 0.03137291627645627), ('chuckles', 0.0278141974152463), ('the', 0.02612848848098895), ('Dustin', 0.025285634013860272)]


We count token prevalence in documents to see if we will benefit from setting the count vectorizer max_df parameter; this little analysis suggests not.

In [6]:
from arrow import now
from bertopic import BERTopic
from plotly.express import histogram
from sklearn.feature_extraction.text import CountVectorizer

# we want to leave proper names intact
LOWERCASE = False
# we tune this to control run time during testing
SAMPLE_FRACTION = 1.0
time_start = now()
large_df = df[['text']].sample(frac=SAMPLE_FRACTION, random_state=2023)
large_model = BERTopic(verbose=True, top_n_words=20, nr_topics=None, language='english', 
                       vectorizer_model=CountVectorizer(stop_words='english', lowercase=LOWERCASE, min_df = 3),
                      embedding_model='all-MiniLM-L6-v2', )
large_topics, large_probs = large_model.fit_transform(large_df['text'].dropna().values)
print('After {} our model says we have {} topics.'.format(now() - time_start, len(set(large_topics))))
print('And we have {} documents with no topic ({}%).'.format(large_topics.count(-1), round(100 * large_topics.count(-1)/len(large_topics))))
histogram(x=[item for item in large_topics if item > -1], title = 'topic > -1 histogram')

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

Batches:   0%|          | 0/334 [00:00<?, ?it/s]

2023-10-25 15:40:21,165 - BERTopic - Transformed documents to Embeddings
2023-10-25 15:40:59,746 - BERTopic - Reduced dimensionality
2023-10-25 15:41:00,197 - BERTopic - Clustered reduced embeddings


After 0:01:38.107212 our model says we have 295 topics.
And we have 1407 documents with no topic (13%).


In [7]:
# What's in that big topic?
large_model.get_topic(topic=0)

[('grunts', 0.1495348420849909),
 ('grunt', 0.05965315594577972),
 ('Tom', 0.022892293487378673),
 ('cool', 0.02065504421426721),
 ('agree', 0.015440032659949051),
 ('tight', 0.015440032659949051),
 ('tomorrow', 0.015440032659949051),
 ('person', 0.015440032659949051),
 ('shouldn', 0.015440032659949051),
 ('strained', 0.015440032659949051),
 ('Nice', 0.014187756633163575),
 ('Try', 0.014187756633163575),
 ('Trust', 0.014187756633163575),
 ('straining', 0.013376487876678857),
 ('Too', 0.013226975478367221),
 ('Son', 0.013226975478367221),
 ('Be', 0.012450386031864023),
 ('hurts', 0.011800754491154876),
 ('Or', 0.011800754491154876),
 ('Ooh', 0.011243921492253397)]

BERTopic will easily visualize all the topics together with a limited number of words per topic, so let's do that.

In [8]:
large_model.visualize_topics(width=1000, height=1000)

And let's pick some words that are important and see what topics/words are nearby in our corpus.

In [9]:
from math import sqrt
from plotly.express import scatter
from sklearn.manifold import TSNE
from plotly.express.colors import sequential


# 1200 seems big but 1000 is cramped
HEIGHT = 1200

def visualize(model, term):
    # let's build a DataFrame of the first 20 topics that include our term
    # and their top terms, along with their weights
    word = []
    topic_indices = []
    topic_probability = []
    word_indices = []
    word_weight = []
    overall_weight = []
    topics, probabilities = model.find_topics(search_term=term, top_n=20)
    for topic_index, topic in enumerate(topics):
        probability = probabilities[topic_index]
        topic_result = model.get_topic(topic=topic)
        for word_index, word_result in enumerate(topic_result):
            word.append(word_result[0])
            topic_indices.append(topic_index)
            topic_probability.append(probability)
            word_indices.append(word_index)
            word_weight.append(word_result[1])
            # let's use the geometric mean to get individual word weights within our sample
            overall_weight.append(sqrt(probability * word_result[1]))
    plot_df = pd.DataFrame(data={'word': word, 'topic_index': topic_indices, 
                                 'topic_probability': topic_probability, 'word_index': word_indices,
                                 'word_weight': word_weight,'overall_weight': overall_weight})
    # terms can recur, so let's get the 'heaviest' occurrence using the overall weight
    plot_df = plot_df.sort_values(ascending=False, by='overall_weight').drop_duplicates(keep='first', subset=['word'])
    # now we want to transform the data we have into something that looks nice in a graph but also
    # keeps the relationships we have from the weight and rank data above
    tsne = TSNE(n_components=2, learning_rate='auto', n_iter=1000, init='pca', random_state=2023, verbose=0)
    plot_df[['x', 'y']] = tsne.fit_transform(X=plot_df.drop(columns=['word']))
    # we use the overall weight for both the color and the size to differentiate the more
    # important words from the others
    scatter(data_frame=plot_df,
            x='x', y='y', text='word', color='overall_weight', size='overall_weight',
            # Shades of red seem appropriate for Stranger Things
            color_continuous_scale=sequential.Reds, title=term, height=HEIGHT,
            # our colorbar doesn't mean anything intuitive so let's remove it
           ).update_traces(textposition='middle right').update_coloraxes(showscale=False).show()

visualize(model=large_model, term='grunt')

In [10]:
for term in ['chuckle', 'Hopper', 'Eleven', 'Dustin', 'Demodog']:
    visualize(model=large_model, term=term)

In [11]:
for term in ['Dustin']:
    visualize(model=large_model, term=term)